<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/climate/climate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install regionmask netCDF4 basemap --quiet

     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 864 kB 54.2 MB/s 
     |████████████████████████████████| 1.0 MB 76.8 MB/s 
     |████████████████████████████████| 19.3 MB 609 kB/s 
     |████████████████████████████████| 6.3 MB 57.6 MB/s 
     |████████████████████████████████| 16.7 MB 44.4 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 30.5 MB 1.2 MB/s 


In [2]:
#import requests
#import json
#import osm2geojson
#import geopandas as gpd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import regionmask
import numpy as np
import PIL
from datetime import datetime, timedelta, date
from mpl_toolkits.basemap import Basemap

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
    [out:json];
    rel["ISO3166-1"~"^HU"]
    [admin_level=2]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("geojson.json", "w")
json.dump(response.json(), output)
output.close()

In [ ]:
data = json.load(open("geojson.json", "r"))
geojson = osm2geojson.json2geojson(data)
gdf = gpd.GeoDataFrame.from_features(geojson)

In [3]:
ds = Dataset("drive/MyDrive/climate/sliced_datasets/tg_ens_mean_0.1deg_reg_1950-1964_v25.0e.nc_sliced")

In [4]:
ds = Dataset("drive/MyDrive/climate/sliced_datasets/tg_ens_mean_0.1deg_reg_1950-1964_v25.0e.nc_sliced")

lats = ds.variables['latitude'][:]
lons = ds.variables['longitude'][:]
dates = ds.variables['time'][:]
rain = ds.variables['tg'][:]

lon, lat = np.meshgrid(lons, lats)

In [ ]:
#longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
start_date = date(1950, 1, 1)

for i in range(365):
  end_date = start_date + timedelta(days=i)
  y, m, d = end_date.strftime("%Y"), end_date.strftime("%m"), end_date.strftime("%d")
  end_date_str = y + "-" + m + "-" + d

  map = Basemap(llcrnrlon=16.06, llcrnrlat=45.65, urcrnrlon=23.02, urcrnrlat=48.63, projection='merc', resolution='i')

  x,y = map(lon, lat)
  map.drawcountries()

  plt.title("Napi átlag hőmérséklet: {0}".format(end_date_str))

  color_scheme = map.pcolormesh(x,y, np.squeeze(rain[i, :, :]), cmap='jet', vmin=-30, vmax=40)
  color_bar = map.colorbar(color_scheme, location='right', pad='5%')
  color_bar.set_label('Napi átlag hőmérséklet (°C)', labelpad=5)
  plt.savefig("drive/MyDrive/climate/avg_temp_output/{0}.jpg".format(i+1), dpi=80)
  plt.clf()


<Figure size 432x288 with 0 Axes>

In [ ]:
frames = []

for i in range(len(dates)):
  frame = PIL.Image.open("drive/MyDrive/climate/avg_temp_output/{0}.jpg".format(str(i+1)))
  frames.append(frame)

frames[0].save('drive/MyDrive/climate/gifs/output.gif', 
               format='GIF',
               append_images = frames[1: ], 
               save_all = True,
               duration=20, loop=0)

In [ ]:
import os


In [ ]:
dir = "drive/MyDrive/climate"
for f in os.listdir(dir):
  fname = os.path.join(dir, f)
  if os.path.isfile(fname):
    ds = xr.open_dataarray(fname)
    ds = ds.sel(time=ds['time'], longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
    ds.to_netcdf("drive/MyDrive/climate/sliced_datasets/{0}_sliced".format(f))
    #print(fname)